# RAG - Triad of metrics

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import utils

import os
import openai
openai.api_key = utils.get_openai_api_key()

In [3]:
from trulens_eval import Tru

tru = Tru()
tru.reset_database()

🦑  initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


In [4]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["./eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()

In [5]:
from llama_index import Document

document = Document(text="\n\n".\
                    join([doc.text for doc in documents]))

In [6]:
from utils import build_sentence_window_index

from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

sentence_index = build_sentence_window_index(
    document,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="sentence_index"
)

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [7]:
from utils import get_sentence_window_query_engine

sentence_window_engine = \
get_sentence_window_query_engine(sentence_index)

config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

In [8]:
output = sentence_window_engine.query(
    "How do you create your AI portfolio?")
output.response

'You create your AI portfolio by showcasing a progression of projects that demonstrate your skills.'

## Feedback functions

In [9]:
import nest_asyncio

nest_asyncio.apply()

In [10]:
from trulens_eval import OpenAI as fOpenAI

provider = fOpenAI()

### 1. Answer Relevance

In [11]:
from trulens_eval import Feedback

f_qa_relevance = Feedback(
    provider.relevance_with_cot_reasons,
    name="Answer Relevance"
).on_input_output()

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .


### 2. Context Relevance

In [12]:
from trulens_eval import TruLlama

context_selection = TruLlama.select_source_nodes().node.text

In [13]:
import numpy as np

f_qs_relevance = (
    Feedback(provider.qs_relevance,
             name="Context Relevance")
    .on_input()
    .on(context_selection)
    .aggregate(np.mean)
)

✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input statement will be set to __record__.app.query.rets.source_nodes[:].node.text .


In [14]:
import numpy as np

f_qs_relevance = (
    Feedback(provider.qs_relevance_with_cot_reasons,
             name="Context Relevance")
    .on_input()
    .on(context_selection)
    .aggregate(np.mean)
)

✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input statement will be set to __record__.app.query.rets.source_nodes[:].node.text .


### 3. Groundedness

In [15]:
from trulens_eval.feedback import Groundedness

grounded = Groundedness(groundedness_provider=provider)

In [16]:
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons,
             name="Groundedness"
            )
    .on(context_selection)
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


## Evaluation of the RAG application

In [17]:
from trulens_eval import TruLlama
from trulens_eval import FeedbackMode

tru_recorder = TruLlama(
    sentence_window_engine,
    app_id="App_1",
    feedbacks=[
        f_qa_relevance,
        f_qs_relevance,
        f_groundedness
    ]
)

In [18]:
eval_questions = []
with open('eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        eval_questions.append(item)

In [19]:
eval_questions

['What are the keys to building a career in AI?',
 'How can teamwork contribute to success in AI?',
 'What is the importance of networking in AI?',
 'What are some good habits to develop for a successful career?',
 'How can altruism be beneficial in building a career?',
 'What is imposter syndrome and how does it relate to AI?',
 'Who are some accomplished individuals who have experienced imposter syndrome?',
 'What is the first step to becoming good at AI?',
 'What are some common challenges in AI?',
 'Is it normal to find parts of AI challenging?']

In [20]:
eval_questions.append("How can I be successful in AI?")

In [21]:
eval_questions

['What are the keys to building a career in AI?',
 'How can teamwork contribute to success in AI?',
 'What is the importance of networking in AI?',
 'What are some good habits to develop for a successful career?',
 'How can altruism be beneficial in building a career?',
 'What is imposter syndrome and how does it relate to AI?',
 'Who are some accomplished individuals who have experienced imposter syndrome?',
 'What is the first step to becoming good at AI?',
 'What are some common challenges in AI?',
 'Is it normal to find parts of AI challenging?',
 'How can I be successful in AI?']

In [22]:
for question in eval_questions:
    with tru_recorder as recording:
        sentence_window_engine.query(question)

In [23]:
records, feedback = tru.get_records_and_feedback(app_ids=[])
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,Groundedness,Context Relevance,Answer Relevance_calls,Groundedness_calls,Context Relevance_calls,latency,total_tokens,total_cost
0,App_1,"{""app_id"": ""App_1"", ""tags"": ""-"", ""metadata"": {...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_3e6560d877829131f5a88bbe6f3bba18,"""What are the keys to building a career in AI?""","""Learning foundational technical skills, worki...",-,"{""record_id"": ""record_hash_3e6560d877829131f5a...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-08-24T07:13:40.777236"", ""...",2024-08-24T07:13:43.051749,1.0,0.500000,0.85,[{'args': {'prompt': 'What are the keys to bui...,[{'args': {'source': 'Chapter 7: A Simple Fram...,[{'args': {'question': 'What are the keys to b...,2,509,0.000779
1,App_1,"{""app_id"": ""App_1"", ""tags"": ""-"", ""metadata"": {...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_29005f8daf4e29c9f0cc266578c444c4,"""How can teamwork contribute to success in AI?""","""Teammates play a crucial role in the success ...",-,"{""record_id"": ""record_hash_29005f8daf4e29c9f0c...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-08-24T07:13:43.482058"", ""...",2024-08-24T07:13:46.039518,1.0,0.566667,NaN,[{'args': {'prompt': 'How can teamwork contrib...,[{'args': {'source': 'To get a project starte...,NaN,2,628,0.000978
2,App_1,"{""app_id"": ""App_1"", ""tags"": ""-"", ""metadata"": {...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_63da3c7dd9fb20da11a841e1693884c6,"""What is the importance of networking in AI?""","""Networking in AI is crucial as it can provide...",-,"{""record_id"": ""record_hash_63da3c7dd9fb20da11a...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-08-24T07:13:46.257639"", ""...",2024-08-24T07:13:49.040111,1.0,0.075000,NaN,[{'args': {'prompt': 'What is the importance o...,[{'args': {'source': 'What is the hiring proce...,NaN,2,539,0.000861
3,App_1,"{""app_id"": ""App_1"", ""tags"": ""-"", ""metadata"": {...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_36430e7b82b607734953dc9e58fc2930,"""What are some good habits to develop for a su...","""Developing good habits in areas such as eatin...",-,"{""record_id"": ""record_hash_36430e7b82b60773495...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-08-24T07:13:49.249809"", ""...",2024-08-24T07:13:51.136778,1.0,NaN,NaN,[{'args': {'prompt': 'What are some good habit...,NaN,NaN,1,434,0.000669
4,App_1,"{""app_id"": ""App_1"", ""tags"": ""-"", ""metadata"": {...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_f1dcf3281797ae06b6c4392761f8b982,"""How can altruism be beneficial in building a ...","""Helping others and aiming to lift them during...",-,"{""record_id"": ""record_hash_f1dcf3281797ae06b6c...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-08-24T07:13:51.396275"", ""...",2024-08-24T07:13:53.362854,1.0,NaN,NaN,[{'args': {'prompt': 'How can altruism be bene...,NaN,NaN,1,482,0.000734


In [24]:
import pandas as pd

pd.set_option("display.max_colwidth", None)
records[["input", "output"] + feedback]

,input,output,Context Relevance,Answer Relevance,Groundedness
0,"""What are the keys to building a career in AI?""","""Learning foundational technical skills, working on projects, finding a job, and being part of a supportive community are the keys to building a career in AI.""",0.85,1.0,0.500000
1,"""How can teamwork contribute to success in AI?""","""Teammates play a crucial role in the success of AI projects. Working collaboratively with colleagues who are dedicated, continuously learning, and focused on building AI for the benefit of all can positively influence individual performance. The ability to work effectively in a team, leverage diverse skills and insights, and collectively steer projects towards success is essential in the field of AI.""",NaN,1.0,0.566667
2,"""What is the importance of networking in AI?""","""Networking in AI is crucial as it can provide valuable insights, guidance, and opportunities for individuals looking to advance in the field. By connecting with professionals who have experience in AI, individuals can gain knowledge about the industry, potential career paths, and current trends. Networking also allows individuals to build relationships with others in the field, which can lead to mentorship, collaboration on projects, and even job opportunities. Additionally, networking can help individuals stay informed about the latest developments in AI and expand their professional connections for future growth and success.""",NaN,1.0,0.075000
3,"""What are some good habits to develop for a successful career?""","""Developing good habits in areas such as eating, exercise, sleep, personal relationships, work, learning, and self-care can help individuals move forward in their careers while maintaining their health.""",NaN,1.0,NaN
4,"""How can altruism be beneficial in building a career?""","""Helping others and aiming to lift them during one's career journey can lead to better outcomes for oneself.""",NaN,1.0,NaN
5,"""What is imposter syndrome and how does it relate to AI?""","""Imposter syndrome is a phenomenon where individuals doubt their accomplishments and have a persistent fear of being exposed as a fraud, despite evidence of their success. In the context of AI, newcomers to the field may experience imposter syndrome, feeling like they do not truly belong in the AI community even if they have achieved success. This can be a common experience for individuals in AI, as highlighted by the mention of imposter syndrome among newcomers to the field.""",NaN,1.0,NaN
6,"""Who are some accomplished individuals who have experienced imposter syndrome?""","""Former Facebook COO Sheryl Sandberg, U.S. first lady Michelle Obama, actor Tom Hanks, and Atlassian co-CEO Mike Cannon-Brookes are some accomplished individuals who have experienced imposter syndrome.""",NaN,NaN,NaN
7,"""What is the first step to becoming good at AI?""","""The first step to becoming good at AI is learning foundational technical skills.""",NaN,NaN,NaN
8,"""What are some common challenges in AI?""","""Some common challenges in AI include keeping up-to-date with rapidly evolving technologies, the iterative nature of AI projects leading to uncertainties in project management, difficulties in finding suitable projects and estimating timelines, collaborating with stakeholders who may lack expertise in AI, and the need for continuous learning throughout one's career.""",NaN,NaN,NaN
9,"""Is it normal to find parts of AI challenging?""","""Yes, it is normal to find parts of AI challenging.""",NaN,NaN,NaN


In [25]:
tru.get_leaderboard(app_ids=[])

,Context Relevance,Answer Relevance,Groundedness,latency,total_cost
app_id,,,,,
App_1,0.85,1.0,0.380556,1.545455,0.000842
